# Automated Document Summarization

## Importing Relevant Libraries

In [4]:
import subprocess
#subprocess.check_call(['python','-m','pip','install','PyPDF2'])
import PyPDF2


In [25]:
import matplotlib
import mglearn
import numpy
import pandas
import pdfminer
import sklearn
import pyLDAvis

In [27]:
from __future__ import print_function

#Relevant Libraries for Loading Data
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

#Relevant Analysis Librariess
import re
import numpy as np
import pandas as pd
import mglearn as mg

#Relevant Modeling Libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

%pylab
%matplotlib inline

#Python LDA Visualization Libraries
import pyLDAvis
import pyLDAvis.sklearn

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


## Loading Data

In [36]:
#Creating a Function to Read the PDF Document and convert into Text
def convert_pdf_to_text(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
        interpreter.process_page(page)
        
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    
    return text

#Reading the PDF Document and saving as lone
lone = convert_pdf_to_text('C:/Users/matth/Documents/Jobs/Travelers/NLP/Automated_Doc_Summarization_NLP/Claim_Req_Travelers_v2_ctrl_p_chrome.pdf')

## 1. Text Exploration

In [9]:
# Print input fields
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1
path='C:/Users/matth/Documents/Jobs/Travelers/NLP/Automated_Doc_Summarization_NLP/Claim_Req_Travelers_McDonald_5_14.pdf'
fp = open(path, 'rb')

parser = PDFParser(fp)
doc = PDFDocument(parser)
fields = resolve1(doc.catalog['AcroForm'])['Fields']
for i in fields:
    field = resolve1(i)
    name, value = field.get('T'), field.get('V')
    print ('{0}: {1}'.format(name, value))


b'Travelers Policy Number': b'6JUB-1E112'
b'Authorized Representative Signature': b'Joseph McDonald'
b'Authorized Representative Name - Printed': b'Joseph McDonald'
b'Date - 1': b'05/14/2021'
b'Producer Signature': b'Joseph McDonald'
b'State Producer License No': b'625864115'
b'Date - 2': b'05/14/2021'
b'Agency': b'Travelers Insurance'
b'Agency Contact': b'James Jamey'
b'Agency Phone Number': b'6078658694'
b'Electronic Signature - Authorized Representative': /'Yes'
b'Electronic Signature - Producer': /'Yes'
b'Additional Information': None
b'Proposed Effective Date': b'05/18/2020'
b'Proposed Expiration Date': b'05/18/2021'
b'CB5': /'No'
b"Claimant's settlement demand": b'1000000.00'
b"Insurer's loss reserve": b'3000000.00'
b"Defendant's offer for settlement": b'7000000'
b'Defense costs paid to date': b'0.00'
b'CB10': /'Yes'
b'Total defense costs paid': None
b'Deductible paid': None
b'Total indemnity paid': None
b'Closed claim or suit': None
b'Incident only': None
b"Today's date": b'05/1

In [7]:
#Exploring just text
lone

" \n\n \nTravelers Casualty and Surety Company of America \n \n\nClaim, Suit, Or Incident Additional Information Request\n\n \n\nImportant Note:  You must report any known claim or suit, or incident, act, error, or omission that may in the future give \nrise to a claim or suit, to your current professional liability carrier before the claim reporting period under that policy \nexpires.  Any claim or suit resulting from any incident, act, error, or omission known before the effective date of any \ninsurance policy issued by Travelers may be excluded from coverage under any such policy whether or not such \nknowledge is disclosed in this additional information request. \n\n \n\n \n\nPLEASE COMPLETE A SEPARATE FORM FOR EACH CLAIM, SUIT, OR INCIDENT \n\n \nTHE INFORMATION BEING REQUESTED IS FOR A CLAIMS‐MADE POLICY. IT IS IMPORTANT THAT YOU READ \nALL OF THE PROVISIONS OF YOUR POLICY CAREFULLY. \nDEFENSE EXPENSES MAY BE INCLUDED WITHIN THE LIMITS OF COVERAGE AND DEDUCTIBLE. \nIMPORTANT NOT

## 2. Data Preprocessing

In [8]:
#Splitting lines
clean_cont = lone.splitlines()

#Exploring the document after splitting lines
clean_cont

[' ',
 '',
 ' ',
 'Travelers Casualty and Surety Company of America ',
 ' ',
 '',
 'Claim, Suit, Or Incident Additional Information Request',
 '',
 ' ',
 '',
 'Important Note:  You must report any known claim or suit, or incident, act, error, or omission that may in the future give ',
 'rise to a claim or suit, to your current professional liability carrier before the claim reporting period under that policy ',
 'expires.  Any claim or suit resulting from any incident, act, error, or omission known before the effective date of any ',
 'insurance policy issued by Travelers may be excluded from coverage under any such policy whether or not such ',
 'knowledge is disclosed in this additional information request. ',
 '',
 ' ',
 '',
 ' ',
 '',
 'PLEASE COMPLETE A SEPARATE FORM FOR EACH CLAIM, SUIT, OR INCIDENT ',
 '',
 ' ',
 'THE INFORMATION BEING REQUESTED IS FOR A CLAIMS‐MADE POLICY. IT IS IMPORTANT THAT YOU READ ',
 'ALL OF THE PROVISIONS OF YOUR POLICY CAREFULLY. ',
 'DEFENSE EXPENSES M

In [9]:
#Removing all Non-Alphabetic Characters from the document
shear = [re.sub("[^a-zA-Z]+", " ", s) for s in clean_cont]

#Removing the unnecessary spaces from the document
shears = [x for x in shear if x != ' ']
shearss = [x for x in shears if x != '']

In [10]:
shear

[' ',
 '',
 ' ',
 'Travelers Casualty and Surety Company of America ',
 ' ',
 '',
 'Claim Suit Or Incident Additional Information Request',
 '',
 ' ',
 '',
 'Important Note You must report any known claim or suit or incident act error or omission that may in the future give ',
 'rise to a claim or suit to your current professional liability carrier before the claim reporting period under that policy ',
 'expires Any claim or suit resulting from any incident act error or omission known before the effective date of any ',
 'insurance policy issued by Travelers may be excluded from coverage under any such policy whether or not such ',
 'knowledge is disclosed in this additional information request ',
 '',
 ' ',
 '',
 ' ',
 '',
 'PLEASE COMPLETE A SEPARATE FORM FOR EACH CLAIM SUIT OR INCIDENT ',
 '',
 ' ',
 'THE INFORMATION BEING REQUESTED IS FOR A CLAIMS MADE POLICY IT IS IMPORTANT THAT YOU READ ',
 'ALL OF THE PROVISIONS OF YOUR POLICY CAREFULLY ',
 'DEFENSE EXPENSES MAY BE INCLUDED WITH

In [11]:
shears

['',
 'Travelers Casualty and Surety Company of America ',
 '',
 'Claim Suit Or Incident Additional Information Request',
 '',
 '',
 'Important Note You must report any known claim or suit or incident act error or omission that may in the future give ',
 'rise to a claim or suit to your current professional liability carrier before the claim reporting period under that policy ',
 'expires Any claim or suit resulting from any incident act error or omission known before the effective date of any ',
 'insurance policy issued by Travelers may be excluded from coverage under any such policy whether or not such ',
 'knowledge is disclosed in this additional information request ',
 '',
 '',
 '',
 'PLEASE COMPLETE A SEPARATE FORM FOR EACH CLAIM SUIT OR INCIDENT ',
 '',
 'THE INFORMATION BEING REQUESTED IS FOR A CLAIMS MADE POLICY IT IS IMPORTANT THAT YOU READ ',
 'ALL OF THE PROVISIONS OF YOUR POLICY CAREFULLY ',
 'DEFENSE EXPENSES MAY BE INCLUDED WITHIN THE LIMITS OF COVERAGE AND DEDUCTIBLE '

In [12]:
shearss

['Travelers Casualty and Surety Company of America ',
 'Claim Suit Or Incident Additional Information Request',
 'Important Note You must report any known claim or suit or incident act error or omission that may in the future give ',
 'rise to a claim or suit to your current professional liability carrier before the claim reporting period under that policy ',
 'expires Any claim or suit resulting from any incident act error or omission known before the effective date of any ',
 'insurance policy issued by Travelers may be excluded from coverage under any such policy whether or not such ',
 'knowledge is disclosed in this additional information request ',
 'PLEASE COMPLETE A SEPARATE FORM FOR EACH CLAIM SUIT OR INCIDENT ',
 'THE INFORMATION BEING REQUESTED IS FOR A CLAIMS MADE POLICY IT IS IMPORTANT THAT YOU READ ',
 'ALL OF THE PROVISIONS OF YOUR POLICY CAREFULLY ',
 'DEFENSE EXPENSES MAY BE INCLUDED WITHIN THE LIMITS OF COVERAGE AND DEDUCTIBLE ',
 'IMPORTANT NOTE NEW YORK DEFENSE EXPE

## 3. Topic Modeling

In [13]:
#Fitting Count Vectorizer on the document with Stop Words
vect = CountVectorizer(ngram_range=(1,1), stop_words='english')
dtm = vect.fit_transform(shearss)

#Document Term Matrix
dtm

<122x304 sparse matrix of type '<class 'numpy.int64'>'
	with 651 stored elements in Compressed Sparse Row format>

In [14]:
#Converting the Document Term Matrix from Count Vectorizer into a Pandas Dataframe
pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())

,acceptance,acknowledge,act,actually,additional,affirm,affixed,agencies,agency,agent,...,violation,virginia,washington,willfully,writing,wrongful,years,yes,york,yyyy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Fitting the Latent Dirichlet Allocation Model on the Document Term Matrix
lda = LatentDirichletAllocation(n_components=5)
lda_dtf = lda.fit_transform(dtm)

#Latent Dirichlet Allocation Model
lda_dtf

array([[0.03333891, 0.03341849, 0.03333933, 0.86633105, 0.03357222],
       [0.02860705, 0.02893485, 0.02857258, 0.88517184, 0.02871369],
       [0.01668475, 0.93297327, 0.01666903, 0.01688054, 0.0167924 ],
       [0.01673366, 0.46586783, 0.01674821, 0.01717309, 0.48347721],
       [0.01668448, 0.93288515, 0.01681705, 0.01694219, 0.01667113],
       [0.02503703, 0.02517688, 0.02505469, 0.02574157, 0.89898983],
       [0.0333377 , 0.03339256, 0.03333803, 0.86640448, 0.03352722],
       [0.02860899, 0.02887967, 0.02876769, 0.88516327, 0.02858038],
       [0.02857559, 0.028825  , 0.02860974, 0.02885647, 0.88513319],
       [0.05000877, 0.05000625, 0.7976826 , 0.05000653, 0.05229585],
       [0.02857624, 0.88529257, 0.02857661, 0.02857493, 0.02897964],
       [0.02306902, 0.02292208, 0.02235529, 0.02222641, 0.90942719],
       [0.0500076 , 0.79953153, 0.05000819, 0.05000553, 0.05044714],
       [0.0182025 , 0.01827173, 0.01818613, 0.01832242, 0.92701722],
       [0.01818554, 0.01821441, 0.

## 4. Topic Extracting

In [16]:
#Extracting 5 Topics from LDA and the most common words in each topic
sorting = np.argsort(lda.components_)[:, ::-1]
features = np.array(vect.get_feature_names())

mg.tools.print_topics(topics=range(5), feature_names=features, sorting=sorting, topics_per_chunk=5, n_words=15)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
presents      claim         signature     claim         information   
knowingly     incident      date          suit          insurance     
payment       suit          electronic    information   false         
loss          insurance     authorized    additional    company       
new           claimant      representativeincident      policy        
person        description   yes           section       misleading    
acceptance    act           proposed      travelers     incomplete    
fraudulent    paid          producer      closed        provided      
claim         date          thousand      company       fines         
benefit       defense       dollars       indemnity     application   
settlement    defrauding    yyyy          complete      crime         
york          defraud       dd            rights        guilty        
civil 

In [17]:
#Sentences within the Topic Model 1
topic_0 = np.argsort(lda_dtf[:,0])[::-1]
for i in topic_0[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

crime and subjects such person to criminal and civil penalties In New York the civil penalty is not to exceed five thousand dollars .

Acceptance box below By doing so you agree that your use of a key pad mouse or other device to check the Electronic Signature and .

OREGON Any person who knowingly presents a false or fraudulent claim for payment of a loss or benefit or who knowingly presents .

KENTUCKY NEW JERSEY NEW YORK OHIO AND PENNSYLVANIA Any person who knowingly and with intent to defraud any .

Acceptance box constitutes your signature acceptance and agreement as if actually signed by you in writing and has the same force .



In [18]:
#Senteces within the Topic Model 2
topic_1 = np.argsort(lda_dtf[:,1])[::-1]
for i in topic_1[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

purpose of defrauding or attempting to defraud the company Penalties may include imprisonment fines denial of insurance and civil .

insurance company or other person files an application for insurance or statement of claim containing any materially false information or .

Important Note You must report any known claim or suit or incident act error or omission that may in the future give .

expires Any claim or suit resulting from any incident act error or omission known before the effective date of any .

FLORIDA Any person who knowingly and with intent to injure defraud or deceive any insurer files a statement of claim or an .



In [19]:
#Senteces within the Topic Model 3
topic_2 = np.argsort(lda_dtf[:,2])[::-1]
for i in topic_2[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

 If you are electronically submitting this document apply your electronic signature to this form by checking the Electronic Signature and .

 ALABAMA ARKANSAS DISTRICT OF COLUMBIA MARYLAND NEW MEXICO AND RHODE ISLAND Any person who .

penalty of a fine of not less than five thousand dollars and not more than ten thousand dollars or a fixed term of .

Proposed Expiration Date mm dd yyyy .

Proposed Effective Date mm dd yyyy .



In [20]:
#Senteces within the Topic Model 4
topic_3 = np.argsort(lda_dtf[:,3])[::-1]
for i in topic_3[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

conceals for the purpose of misleading information concerning any fact material thereto commits a fraudulent insurance act which is a .

Provide enough information to allow evaluation use additional information section if necessary DO NOT attach a copy .

This area may be used to provide additional information to any question Reference section name and question number .

therein are deemed fully incorporated herein I also affirm that any declarations made in the core application regarding the .

I acknowledge that this document is to be read in conjunction with the core application and that all notices contained .



In [21]:
#Senteces within the Topic Model 5
topic_4 = np.argsort(lda_dtf[:,4])[::-1]
for i in topic_4[:5]:
    print(f".".join(shearss[i].split(f".")[:2]) + f".\n")

damages Any insurance company or agent of an insurance company who knowingly provides false incomplete or misleading facts or .

COLORADO It is unlawful to knowingly provide false incomplete or misleading facts or information to an insurance company for the .

misleading information to an insurance company for the purpose of defrauding the company Penalties include imprisonment fines .

presents false information in an application for insurance is guilty of a crime and may be subject to fines and confinement in prison .

Answer each question on behalf of all entities seeking insurance coverage unless specifically requested otherwise .



## 5. Topic Visualization

In [22]:
zit=pyLDAvis.sklearn.prepare(lda,dtm,vect)

pyLDAvis.display(zit)